On the sdf login node
```
[youtsumi@sdf-login02 u]$ source /cvmfs/sw.lsst.eu/linux-x86_64/lsst_distrib/w_2021_51/loadLSST.bash                                                                                                        [89/89]
(lsst-scipipe-0.7.0-ext) [youtsumi@sdf-login02 u]$ setup lsst_distrib
(lsst-scipipe-0.7.0-ext) [youtsumi@sdf-login02 u]$ jupyter lab
```

Needs an ssh tunnel in a separate window

In [14]:
! eups list -s lsst_distrib

   g7ae64ea4b3+cca76d10ca 	w_latest w_2021_52 current setup


In [15]:
import sys
sys.path.append("/sdf/group/lsst/camera/IandT/repo_gen3/spot_test_v2/u/youtsumi/202112/mixcoatl/python")

In [16]:
import numpy
import pylab as plt

# Stack imports
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
import lsst.meas.extensions.shapeHSM
import lsst.afw.image as afwImage
from lsst.ip.isr.isrTask import IsrTask
from mixcoatl.characterizeSpots import CharacterizeSpotsTask
from mixcoatl.gridFitTask import GridFitTask

import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [17]:
def display(exposure,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('asinh', 'zscale')
    afw_display.mtv(exposure.getImage())
    plt.title(title)
    plt.gca().axis('off')

In [18]:
repo_path = "/sdf/group/lsst/camera/IandT/repo_gen3/BOT_data/"
butler = Butler(repo_path)

In [19]:
collections="u/youtsumi/Run5/run_R03_S12_dither/spot"
butler = Butler(repo_path,collections=collections)
registry = butler.registry

In [20]:
print(registry)

PostgreSQL@rubin:bot_data_w_2021_43


In [21]:
butler = Butler(repo_path)
registry = butler.registry
for c in registry.queryCollections():
    print(c)    

LSSTCam/calib
LSSTCam/calib/unbounded
LSSTCam/raw/all
LSSTCam/photodiode/all
u/snyder18/crosstalk_13169/sbias
u/snyder18/crosstalk_13169/sbias/20211215T191900Z
u/snyder18/crosstalk_13169/calib
u/snyder18/crosstalk_13169/crosstalk_spot
u/snyder18/crosstalk_13175/sbias
u/snyder18/crosstalk_13175/sbias/20211217T211636Z
u/snyder18/crosstalk_13186/sbias
u/snyder18/crosstalk_13186/sbias/20211217T225138Z
u/snyder18/crosstalk_13175/calib
u/snyder18/crosstalk_13169/crosstalk_spot/20211218T003408Z
u/snyder18/crosstalk_13175/crosstalk_spot
u/snyder18/crosstalk_13186/calib
u/snyder18/crosstalk_13175/crosstalk_spot/20211218T225100Z
u/snyder18/crosstalk_13186/crosstalk_spot
u/snyder18/crosstalk_13186/crosstalk_spot/20211220T023605Z
u/snyder18/crosstalk_13198/sbias
u/snyder18/crosstalk_13198/sbias/20211220T182307Z
u/snyder18/crosstalk_13198/calib
u/snyder18/crosstalk_13200/sbias
u/snyder18/crosstalk_13200/sbias/20211220T215856Z
u/snyder18/crosstalk_13200/calib
u/snyder18/crosstalk_13200/crosstalk_spo

In [22]:
# then explore the inside of the collection
subbutler = Butler(repo_path,collections=collections)
subregistry = subbutler.registry
for dtype in subregistry.queryDatasetTypes():
    try:
        datasetRefs = list(subregistry.queryDatasets(datasetType=dtype,collections=collections))        
        print(len(datasetRefs), dtype )
    except:
        print("Error", dtype )

0 DatasetType('camera', {instrument}, Camera, isCalibration=True)
3253972 DatasetType('raw', {band, instrument, detector, physical_filter, exposure}, Exposure)
0 DatasetType('photodiode', {band, instrument, physical_filter, exposure}, AstropyTable)
0 DatasetType('cpBiasProc', {band, instrument, detector, physical_filter, exposure}, Exposure)
4996 DatasetType('isr_metadata', {band, instrument, detector, physical_filter, exposure}, PropertySet)
0 DatasetType('bias', {instrument, detector}, ExposureF, isCalibration=True)
0 DatasetType('cpCombine_metadata', {instrument, detector}, PropertySet)
3 DatasetType('isr_config', {}, Config)
0 DatasetType('cpCombine_config', {}, Config)
3 DatasetType('packages', {}, Packages)
0 DatasetType('cpCrosstalkProc', {band, instrument, detector, physical_filter, exposure}, Exposure)
0 DatasetType('crosstalkBackgroundZOffsets', {band, instrument, detector, physical_filter, exposure}, StructuredDataDict)
0 DatasetType('crosstalkBackgroundYTilts', {band, instr

In [23]:
where = """
instrument='LSSTCam'
and detector=23
"""
#and exposure.observation_type = 'spot'
#and exposure.science_program IN ('13232','13234')

datasetRefs = list(subregistry.queryDatasets(
    datasetType="gridSpotSrc",
    collections=collections,
    where=where)
                  )
results = []

for aref in datasetRefs[::]:
    try:
        # get a catalog from the butler
        src = subbutler.get(aref)
        # get the associated metadata from raw data which contains EXPTIME
        raw = subbutler.get("raw",dataId=aref.dataId)
        img = raw.getImage().getArray()
        md = raw.getMetadata()
        # stores into memory for later use
        results.append( {"md": md, "src":src} )
    except:
#        raise
        pass

In [24]:
from astropy.table import vstack, join
faint = []
for aresult in results:
    if aresult["md"]["EXPTIME"]>13:
        continue
    faint.append(aresult["src"].asAstropy())
mean_data_table = vstack(faint, join_type='exact')

mean_data_table_by_groups = mean_data_table[ numpy.where(
        (mean_data_table["spotgrid_index"]>=0) &
        (mean_data_table["base_PixelFlags_flag"]==False) &
        (mean_data_table["base_PsfFlux_flag"]==False) &
        (mean_data_table["base_SdssCentroid_flag"]==False) &
        (mean_data_table["base_CircularApertureFlux_70_0_flag_apertureTruncated"]==False) &
        (mean_data_table["spotgrid_x"]<3800) &
        (mean_data_table["spotgrid_y"]<3800) &
        (mean_data_table["spotgrid_x"]>200) &
        (mean_data_table["spotgrid_y"]>200)
    )].group_by('spotgrid_index')
ref = mean_data_table_by_groups.groups.aggregate(numpy.nanmedian)
ref

ValueError: no values provided to stack.

In [ ]:
exparray = numpy.array([ aresult["md"]["EXPTIME"] for aresult in results ])
exptime=list(set(exparray))

In [ ]:
exparray[ numpy.argmin(exparray) ]

In [ ]:
fontsize=14
fig = plt.figure(figsize=(10,10),facecolor="white")

#ykey="ext_shapeHSM_HsmShapeRegauss_sigma"
#ykey="base_SdssShape_xx"
ykey="ext_shapeHSM_HsmSourceMoments_xx"
#y2key="base_SdssShape_yy"
y2key="ext_shapeHSM_HsmSourceMoments_yy"
xkey="base_CircularApertureFlux_50_0_instFlux"
#xkey="base_SdssShape_instFlux"
x = []
y = []
e = []
for aresult in results[:]:
#    cond = ( aresult["src"]["spotgrid_index"]>=0 ) & ( aresult["src"]["spotgrid_normalized_dx"]**2+aresult["src"]["spotgrid_normalized_dy"]**2<0.002**2 )
    cond = ( aresult["src"]["spotgrid_index"]>=0 ) & (aresult["src"]["base_CircularApertureFlux_70_0_instFlux"]>numpy.nanmax(aresult["src"]["base_CircularApertureFlux_70_0_instFlux"])*0.6)
    x.extend(aresult["src"][xkey][cond])
    y.extend((aresult["src"][ykey]+aresult["src"][y2key])[cond])
    e.extend([aresult["md"]["EXPTIME"]]*len(aresult["src"][xkey][cond]))
x=numpy.array(x)
y=numpy.array(y)
plt.scatter(x,y,c=e,s=3,cmap='turbo',alpha=1)
plt.xlabel(xkey,fontsize=fontsize)
plt.ylabel("{}".format("T=I_ii where I_ii=ext_shapeHSM_HsmSourceMoments_ii"),fontsize=fontsize)
plt.xlim(1e4,0.4e7)
plt.ylim(9.2,10.2)
#plt.ylim(1e4,1e8)
plt.colorbar(label="EXPTIME")
#plt.loglog()
#plt.semilogx()


In [ ]:
from astropy.table import vstack, join
faint = [ results[i]["src"].asAstropy() for i in  numpy.where(exparray == exptime[2] )[0] ] ### this determines which one from the faintest we are going to base on
mean_data_table = vstack(faint, join_type='exact')

mean_data_table_by_groups = mean_data_table[ numpy.where(
        (mean_data_table["spotgrid_index"]>=0)
    )].group_by('spotgrid_index')
ref = mean_data_table_by_groups.groups.aggregate(numpy.nanmedian)
ref

In [ ]:
#pylab.scatter(
#    results[0]["src"]["spotgrid_x"],
#    results[0]["src"]["spotgrid_y"]
#)

fig = plt.figure(figsize=(10,10))

for i in range(100):
    pylab.scatter(
        results[i]["src"]["spotgrid_y"],
        results[i]["src"]["spotgrid_x"]
    )

#spotgrid_y	spotgrid_x	spotgrid_normalized_dy	spotgrid_normalized_dx	


In [ ]:
fig = plt.figure(figsize=(10,10),facecolor="white")

from astropy.table import vstack, join

y1key="ext_shapeHSM_HsmSourceMoments_xx"
y2key="ext_shapeHSM_HsmSourceMoments_yy"
#xkey="base_CircularApertureFlux_70_0_instFlux"
xkey="base_CircularApertureFlux_70_0_instFlux"
zskey="base_CircularApertureFlux_3_0_instFlux"
#zskey="base_PsfFlux_instFlux"
zlkey="base_CircularApertureFlux_70_0_instFlux"
# ext_shapeHSM_HsmSourceMoments_xx
  
x = []
y = []
zl = []
zs = []
e = []
for aresult in results[:]:
    joined = join(aresult["src"].asAstropy(),ref,keys="spotgrid_index")
    Iperr = joined["base_PsfFlux_instFluxErr_1"]
    Ip = joined["base_PsfFlux_instFlux_1"]    
    Ierr = joined["{}Err_1".format(xkey)]
    I = joined["{}_1".format(xkey)]
    median=numpy.nanmedian(joined["base_CircularApertureFlux_70_0_instFlux_1"])
    joined = joined[
        numpy.where( 
#            (joined["base_PixelFlags_flag_1"] == False) &
#            (joined["base_PsfFlux_flag_edge_1"] == False) &
#            (joined["base_CircularApertureFlux_70_0_flag_apertureTruncated_1"] == False) &            
#            (joined["base_ClassificationExtendedness_flag_1"] == False) &
#            (joined["base_PixelFlags_flag_1"] == False) &            
#            (joined["base_PixelFlags_flag_offimage_1"] == False) &
#            (joined["base_PixelFlags_flag_edge_1"] == False) &
#            (joined["base_PixelFlags_flag_interpolated_1"] == False) &
#            (joined["base_PixelFlags_flag_saturated_1"] == False) &            
#            (joined["base_PixelFlags_flag_cr_1"] == False) &
#            (joined["base_PixelFlags_flag_bad_1"] == False) &
#            (joined["base_PixelFlags_flag_suspect_1"] == False) &
#            (joined["base_PixelFlags_flag_interpolatedCenter_1"] == False) &
#            (joined["base_PixelFlags_flag_saturatedCenter_1"] == False) &
#            (joined["base_PixelFlags_flag_crCenter_1"] == False) &
#            (joined["base_PixelFlags_flag_suspectCenter_1"] == False) &
            ((joined["base_CircularApertureFlux_70_0_instFlux_1"]-median)/median<0.01)
#            (Iperr/Ip<0.01)&
#            (Ierr/I<0.01)            
        )
    ]
    x.extend(joined["{}_1".format(xkey)])
    Y1=joined["{}_1".format(y1key)]
    Y2=joined["{}_1".format(y2key)]
    Y0_1=joined["{}_2".format(y1key)]
    Y0_2=joined["{}_2".format(y2key)]    
    zl.extend(joined["{}_1".format(zlkey)]/joined["{}_2".format(zlkey)])
    zs.extend(joined["{}_1".format(zskey)]/joined["{}_2".format(zskey)])
    y.extend( Y1+Y2-(Y0_1+Y0_2) )
    e.extend([aresult["md"]["EXPTIME"]]*len(Y1))
    
x=numpy.array(x)
y=numpy.array(y)
zl=numpy.array(zl)
zs=numpy.array(zs)

fig = plt.figure(figsize=(10,20),facecolor="white")

ax = plt.subplot(211)
sca=ax.scatter(x,y,c=e,s=5,cmap='turbo',alpha=0.5)
ax.set_ylabel("{}".format(r"$dT=T-T_0$, ext_shapeHSM_HsmSourceMoments_ii"),fontsize=fontsize)
ax.set_xlim(1e5,1e7)
ax.set_ylim(-1,1)
ax.grid()

bx = plt.subplot(212,sharex=ax)
scb=bx.scatter(x,zs/zl-1,c=e,s=5,cmap='turbo',alpha=0.5)
bx.set_ylabel("{}".format(r"$(I_{70}/I_{\rm psf})/(I_{70}/I_{\rm psf})_0-1$, CircularAperture"),fontsize=fontsize)
bx.set_xlim(1e5,0.55e7)
bx.set_ylim(-.3,.3)
#plt.semilogx()
bx.grid()
bx.set_xlabel(xkey,fontsize=fontsize)

#plt.colorbar(scb,label="EXPTIME",orientation="horizontal",location="bottom")
cbar_ax = fig.add_axes([0.97, 0.15, 0.03, 0.7])
plt.colorbar(scb,cax=cbar_ax,label="EXPTIME")

plt.subplots_adjust(right=0.93,hspace=0)

In [ ]:
import pylab
fig = plt.figure(figsize=(10,10))

#for a in faint:
peak = []
exp = []
run = []
for b in results:
    peak.append(numpy.nanmax(b["src"]["base_CircularApertureFlux_70_0_instFlux"]))
    exp.append(b["md"]["EXPTIME"])
    run.append(b["md"]["RUNNUM"])

exp=numpy.array(exp)
peak=numpy.array(peak)
run=numpy.array(run)
for arun in list(set(run)):
    print(numpy.where(run == arun))
    pylab.plot(exp[numpy.where(run == arun)],peak[numpy.where(run == arun)],"o",label=arun)
pylab.legend()
pylab.ylabel("base_CircularApertureFlux_70_0_instFlux")
pylab.xlabel("EXPTIME [s]")

In [ ]:
where = """
instrument='LSSTCam'
and detector=23
and exposure.observation_type = 'spot'
and exposure.exposure_time = 1.25
and exposure.science_program IN ('13232','13234')
"""

# gridSpotSrc

datasetRefs = list(subregistry.queryDatasets(
    datasetType="postISRCCD",
    collections=collections,
    where=where)
                  )
subresults = []

for aref in datasetRefs[:5]:
    try:
        # get a catalog from the butler
        raw = subbutler.get(aref)
        # get the associated metadata from raw data which contains EXPTIME
        src = subbutler.get("gridSpotSrc",dataId=aref.dataId)
        img = raw.getImage().getArray()
#        print(src.asAstropy()["base_SdssCentroid_x","base_SdssCentroid_y","base_SdssShape_instFlux"])
        md = raw.getMetadata()
        x=src.asAstropy()["spotgrid_x"]
        y=src.asAstropy()["spotgrid_y"]
        I=src.asAstropy()["base_SdssShape_instFlux"]        
        display(raw)
        pylab.title(f"{md['OBSID']} {md['RAFTBAY']}/{md['CCDSLOT']}")
        pylab.plot(x,y,"o")        
        pylab.show()

        print(md["EXPTIME"])
        # stores into memory for later use
        subresults.append( {"md": md, "src":src} )
    except:
#        raise
        pass


In [ ]:
list(md)